In [1]:
import pandas as pd
import numpy as np

In [2]:
file_name = 'all_info.csv'

In [3]:
allinfotypes = {"date":str,
                "PX_LAST":float,
                "PX_OPEN":float,
                "PX_HIGH":float,
                "PX_LOW":float,
                "PX_VOLUME":float,
                "TOT_RETURN_INDEX_NET_DVDS":float,
                "etf":str}
rawdata = pd.read_csv(file_name,sep =",",encoding = 'UTF-8', dtype = allinfotypes )

In [4]:
rawdata["date"] = pd.to_datetime(rawdata["date"],format = '%Y-%m-%d')

In [5]:
columns_relevant = ["date","TOT_RETURN_INDEX_NET_DVDS","etf"]
datafil = rawdata[columns_relevant].copy()

In [6]:
datafilind = datafil.set_index(['date','etf']).unstack()
cols = [f"{l1}" for (l0, l1) in datafilind.columns]
datafilind.columns = cols
datafilind.reset_index(inplace = True)

In [7]:
fill_date = (datafilind['date']>'1994-01-01')

In [8]:
datafilind.columns

Index(['date', 'AAXJ US Equity', 'EIDO US Equity', 'EPHE US Equity',
       'EWA US Equity', 'EWD US Equity', 'EWG US Equity', 'EWH US Equity',
       'EWI US Equity', 'EWL US Equity', 'EWM US Equity', 'EWN US Equity',
       'EWP US Equity', 'EWQ US Equity', 'EWS US Equity', 'EWT US Equity',
       'EWU US Equity', 'EWY US Equity', 'EZU US Equity', 'IEV US Equity',
       'INDA US Equity', 'MCHI US Equity', 'PAK US Equity', 'SPY US Equity',
       'THD US Equity', 'XLB US Equity', 'XLC US Equity', 'XLE US Equity',
       'XLF US Equity', 'XLI US Equity', 'XLK US Equity', 'XLP US Equity',
       'XLU US Equity', 'XLY US Equity'],
      dtype='object')

In [9]:
etfs = ['EWI US Equity','EWD US Equity']

In [10]:
data_to_prepros = datafilind[['date']+etfs][fill_date].copy()

In [11]:
data_to_prepros.set_index('date',inplace = True)

## Creación de las caracteristicas

In [12]:
data_to_prepros.sort_index(inplace = True)
for i in etfs:
    name_col = i +'_cp_precio'
    data_to_prepros[name_col] =data_to_prepros[[i]].rolling(14, min_periods=14).mean()/data_to_prepros[[i]]
    
    name_col = i +'_mp_precio'
    data_to_prepros[name_col] =data_to_prepros[[i]].rolling(50, min_periods=50).mean()/data_to_prepros[[i]]
    
    name_col = i +'_lp_precio'
    data_to_prepros[name_col] =data_to_prepros[[i]].rolling(200, min_periods=200).mean()/data_to_prepros[[i]]
    
    name_col = i +'_cp_mp'
    data_to_prepros[name_col] =data_to_prepros[[i]].rolling(14, min_periods=14).mean()/data_to_prepros[[i]].rolling(50, min_periods=50).mean()
    
    name_col = i +'_cp_lp'
    data_to_prepros[name_col] =data_to_prepros[[i]].rolling(14, min_periods=14).mean()/data_to_prepros[[i]].rolling(200, min_periods=200).mean()

    name_col = i +'_mp_lp'
    data_to_prepros[name_col] =data_to_prepros[[i]].rolling(50, min_periods=50).mean()/data_to_prepros[[i]].rolling(200, min_periods=200).mean()

    name_col = i +'_compare'
    data_to_prepros[name_col] =data_to_prepros[[i]].shift(periods=-5)/data_to_prepros[[i]]

In [13]:
data_to_prepros['CLASS']= (data_to_prepros['EWI US Equity_compare']>data_to_prepros['EWD US Equity_compare']).astype(int)

In [14]:
data_to_prepros.dropna(inplace = True)
data_to_prepros.head()

,EWI US Equity,EWD US Equity,EWI US Equity_cp_precio,EWI US Equity_mp_precio,EWI US Equity_lp_precio,EWI US Equity_cp_mp,EWI US Equity_cp_lp,EWI US Equity_mp_lp,EWI US Equity_compare,EWD US Equity_cp_precio,EWD US Equity_mp_precio,EWD US Equity_lp_precio,EWD US Equity_cp_mp,EWD US Equity_cp_lp,EWD US Equity_mp_lp,EWD US Equity_compare,CLASS
date,,,,,,,,,,,,,,,,,
2000-06-06,63.8219,40.8844,0.944053,0.944036,0.929661,1.000018,1.015481,1.015463,1.024629,0.934480,0.958419,0.841988,0.975022,1.109849,1.138281,0.957720,1
2000-06-07,64.2935,40.8844,0.943415,0.936867,0.923592,1.006990,1.021463,1.014373,1.026894,0.938550,0.957868,0.843718,0.979833,1.112398,1.135294,0.952207,1
2000-06-08,63.9791,40.2832,0.954194,0.941225,0.928883,1.013779,1.027249,1.013287,1.019655,0.954956,0.971230,0.858043,0.983244,1.112947,1.131913,0.944029,1
2000-06-09,64.1363,40.1329,0.960609,0.939114,0.927354,1.022888,1.035860,1.012681,1.012254,0.963616,0.974081,0.862979,0.989256,1.116616,1.128743,0.936329,1
2000-06-12,64.4507,40.3583,0.965156,0.934972,0.923562,1.032284,1.045037,1.012354,1.007316,0.967146,0.968678,0.859899,0.998418,1.124720,1.126502,0.925512,1


In [15]:
data_to_prepros.to_csv('EWI_US_EWD_US.csv', decimal = '.',sep = ';')